<a href="https://colab.research.google.com/github/phylars/ILEE_CSK/blob/ipynb/ILEE_2D_mode_(V_0_1_0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparation**

Please transfer you confocal format files to raw (stack) tiff images using ImageJ/FIJI, and put them in a new folder. 
*   If you wish to run the pipeline in Google Colab, please upload the tiff folder to your Google Drive; 
*   If you wish to run the pipeline locally, please download this .ipynb file to your local device and open it by Jupyter Notebook. 


In [ ]:
#@title Install ILEE_CSK and dependencies
!pip install -U ILEE_CSK 
import ILEE_CSK 
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scikit_posthocs as sp

In [ ]:
#@title Inspect the data structure of your tif image
#@markdown All your image samples should be taken under the same confocal setting. Please input the file path of any one of your image file in this batch of experiment, below. It can be local (for jupyter notebook) or in your Google Drive (Google Colab):

example_tif_file_path = "example_document/example-123.tif"  #@param {type: "string"}
img_example = io.imread(example_tif_file_path)
print('The image array structure is', img_example.shape)

#@markdown (Optional/Recommended): This pipeline also provide data visualization and significance analysis. To enable this, please name your tiff files as (group_name)-(repeat_index).tif, such as mock-1.tif, mock-2.tif, or treatment1-1.tif ... The hyphen '-' must occur only once. 

The image array structure needs to meet 2 requirements:

*   Your image array structure should be 3D or 4D (seeing 3 or 4 numbers in the tuple). If it is 2D, you may be using a projected image, ILEE_CSK fully supports it but it cannot be processed in this pipeline. If it has 5 or more dimension, this is an abnormal data structure and please submit a bug report in the Github.
*   If it is 4D, the dimension with lowest size should be your channel, and the the dimension with second lowest size should be your z-axis.

To learn which channel is your objective, please run the following step to inspect your example image:

In [ ]:
#@title Inspect the channels of the example image
def align_image_dimension(img):
    img_size = np.array(img.shape)
    dimension = img_size.shape[0]
    df_dimension = pd.DataFrame(data = img_size, columns = ['dimension_size'])
    df_dimension['dimension_index'] = range(img_size.shape[0])
    df_dimension = df_dimension.sort_values(by = ['dimension_size', 'dimension_index'])
    if (dimension <= 2):
      print ('Error: image have less than 3 dimensions; please check data structure')
      return()
    elif (dimension == 3):
      result = np.moveaxis(img, source = np.array(df_dimension['dimension_index']), destination = np.array([0,1,2]))
    elif (dimension == 4):
      result = np.moveaxis(img, source = np.array(df_dimension['dimension_index']), destination = np.array([0,1,2,3]))
    else:
      print('image has more than 4 dimension (x, y, z, and channel); please check data structure')
      return()
    return (result)

if(len(img_example.shape) == 3):
  print('Your image sample is a 3D array (does not have channel dimension). Please set objective_channel to None in the next step.')
  print('\n')
  img_example = align_image_dimension(img_example)
  img_example_p = np.amax(img_example, axis=0)
  print('Below is a z-axis projected of your image sample:')
  plt.figure(figsize = (6, 6))
  plt.imshow(img_example_p)
  plt.axis('off')
  plt.show()
elif(len(img_example.shape) == 4):
  print('Your image sample is a 4D array (contains channel dimension). The projection image of each channel is shown below:')
  print('\n')
  img_example = align_image_dimension(img_example)
  for i in range(img_example.shape[0]):
    img_example_p = np.amax(img_example[i,:,:,:], axis=0)
    print('channel', i)
    plt.figure(figsize = (6, 6))
    plt.imshow(img_example_p)
    plt.axis('off')
    plt.show()
    print('\n')
print('Please set the objective_channel to the correct channel index for cytoskeleton in the next step.')

In [ ]:
#@title Input the image folder containing all the raw tiffs in this experiment

folder_path = 'example_folder/example_subfolder'  #@param {type: "string"}

In [ ]:
#@title Estimate optimal K2
#@markdown K2 is an parameter required for ILEE processing. We recommend use the optimal K2 calculated by this function for most of the cases.

#@markdown According to the projected images of each channel, shown above, please input channel index of cytoskeleton fluorescence:

objective_channel_index = 0  #@param {type: "number"}
optimal_K2 = ILEE_CSK.opt_k2(folder_path, target_channel = objective_channel_index)
print('The estimated optimal K2 is', optimal_K2)

# **Start processing your samples**

In [ ]:
#@title Set input parameters and run

objective_channel_index = 0  #@param {type: "number"}
#@markdown Highly recommended set K1 to 2.5, unless you know what it means.
K1 = 2.5  #@param {type: "number"}
#@markdown Copy your optimal K2 to this place:
K2 =   Input_your_K2_here#@param {type: "number"}
#@markdown Highly recommended use pL_8.
pL_type = "pL_8"  #@param ['pL_8', 'pL_4']
exclude_true_blank = False  #@param {type:"boolean"}
pixel_size = 1  #@param {type: "number"}

df_result = ILEE_CSK.analyze_document_2D (folder_path = folder_path,
                                          obj_channel = objective_channel_index,
                                          k1 = K1,
                                          k2 = K2,
                                          pixel_size = 1, 
                                          exclude_true_blank = exclude_true_blank)





In [ ]:
#@title Show your data table
df_result.style

In [ ]:
#@title Download the data as Excel file.
from google.colab import files
df_result.to_excel('result.xlsx')
files.download('result.xlsx')

In [ ]:
#@title Visualize your data
#@markdown As noted above, to enable visualization and significance test, your tiff files must be named as "xxx(group)-xxx(index)".
import seaborn as sns

df_result['group'] = [i.split('-')[0] for i in df_result['file_name']]
df_result['repeat'] = [(i.split('-')[1]).split('.')[0] for i in df_result['file_name']]

indices = df_result.columns[0:9]

sns.set(font_scale = 1.6)
sns.set_style("ticks")
fig, axes = plt.subplots(3, 3, figsize=(16,16))
for i in range(9):
  g1 = sns.barplot(data = df_result,
                   x = 'group', 
                   y = indices[i], 
                   ci = 'sd',
                   errwidth = 4,
                   capsize= 0.3,
                   errcolor = 'black',
                   palette = 'deep',
                   ax = axes[int(i/3),i%3])
  g2 = sns.stripplot(data = df_result, 
                    x = 'group', 
                    y = indices[i],
                    size = 16, 
                    edgecolor = 'gray', 
                    linewidth = 3, 
                    alpha = 0.86,
                    palette = 'dark',
                    ax = axes[int(i/3),i%3])
fig.tight_layout()
plt.show()




In [ ]:
#@title Significant test

p_thres = 0.05  #@param {type: "number"}
Data_property = "I am measuring biological samples and/or want to learn the biological features at my experiment condition. (Use post-hoc)"  #@param ['I am measuring biological samples and/or want to learn the biological features at my experiment condition. (Use post-hoc)', 'I am not using data to interpret biological features and the image per-se is ground truth. (No post-hoc)']
if (Data_property == 'I am measuring biological samples and/or want to learn the biological features at my experiment condition. (Use post-hoc)'):
  p_adjust = 'holm'
else:
  p_adjust = None

print('Testing with P value threshold at', p_thres)
print('Family-wise error correction :', p_adjust)
print('\n')

def multiple_comparison (df, cat_var_column, value_column, p_thres, p_adjust = p_adjust):
    #cat_vars: list of names of columns for categorical variables
    #value: name of (dependent) the numeric variable you want to compare
    result = pd.DataFrame()
    cat_vars = np.unique(df[cat_var_column])
    n = cat_vars.shape[0]
    
    data = []
    for i in range(n):
        data.append(df[df[cat_var_column] == cat_vars[i]][value_column])
        new_row = pd.DataFrame(columns = ['group','index','n','mean','std'],
                               data = [[cat_vars[i], i, data[i].shape[0], np.mean(data[i]), np.std(data[i])]])
        result = result.append(new_row, ignore_index=True)     
    p_matrix = sp.posthoc_ttest(data, p_adjust = p_adjust)
    p_matrix = np.array(p_matrix)
    
    result = result.sort_values(by = ['mean'], ascending = False, ignore_index = True)
    result['mark'] = ''
    
    start=0
    mark=0
    result.at[0, 'mark'] = chr(97+mark)
    
    while(1):
        for i in range(start + 1, n):
            #print ('forward comparing', start, i, chr(97+mark))
            start_id = result.loc[start]['index']
            i_id = result.loc[i]['index']
            #print(p_matrix[start_id,i_id])
            if (p_matrix[start_id,i_id] > p_thres):
                if (result.at[i, 'mark'] == ''):
                  result.at[i, 'mark'] = result.at[i, 'mark'] + (chr(97+mark))
                elif (result.at[i, 'mark'][-1] != chr(97+mark)):
                  result.at[i, 'mark'] = result.at[i, 'mark'] + (chr(97+mark))
            else:
                mark+=1
                result.at[i, 'mark'] = result.at[i, 'mark'] + (chr(97+mark))
                start = i*1
                #print (i, 'extend', chr(97+mark))
                break
        for ii in range (i-1, 0, -1):
            #print ('backward comparing', i, ii, chr(97+mark))
            ii_id = result.loc[ii]['index']
            i_id = result.loc[i]['index']
            if (p_matrix[ii_id, i_id] > p_thres):
                if (result.at[ii, 'mark'] == ''):
                  result.at[ii, 'mark'] = result.at[ii, 'mark'] + (chr(97+mark))
                elif (result.at[ii, 'mark'][-1] != chr(97+mark)):
                  result.at[ii, 'mark'] = result.at[ii, 'mark'] + (chr(97+mark))
        if (i==n-1):
            break
        
    return (result)
            
def df_multiple_comparison (df, cat_var_column, p_thres = p_thres, column_types = ['float'], p_adjust = p_adjust):
    sd_table = pd.DataFrame(columns = ['group'], data = np.unique(df[cat_var_column]))
    value_cols = []
    for col in df.columns:
      if (df[col].dtype in column_types):
        value_cols.append(col)
    for col in value_cols:
      result = multiple_comparison(df = df, cat_var_column = cat_var_column, value_column = col, p_thres = p_thres, p_adjust = p_adjust)
      print('Cytoskeleton index:', col)
      print(result, '\n')
      add_on_table = result[['group', 'mark']]
      add_on_table = add_on_table.rename(columns={'mark': col})
      sd_table = sd_table.merge(add_on_table, on = 'group')
    return (sd_table)

sd_table = df_multiple_comparison(df = df_result, cat_var_column = 'group', p_thres = p_thres, p_adjust = p_adjust)
print('Comparison of all indices:')
sd_table.style